In [6]:
import requests

# Scraping all F2024 Courses

all_courses = requests.get("https://penncourseplan.com/api/base/current/courses").json()

In [27]:
# List for sections
section_list = []

# List for courses
course_list = []

# Iterate through all courses
for course in all_courses:

    # proceed if at least 1 section

    if course["num_sections"] > 0:
        
        # Grab course data for ID
        
        id = course["id"]
        course_data = requests.get("https://penncourseplan.com/api/base/current/courses/" + id + "/").json()

        # Data common to course and section lists
        common = [course_data["id"], course_data["title"], course_data["description"], course_data["prerequisites"]]

        # Data unique to course list
        course_additions = [course_data["course_quality"], course_data["instructor_quality"], course_data["difficulty"], course_data["work_required"], course_data["credits"]]

        # Iterating through available sections
        for section in course_data["sections"]:
            
            instructor = ""
            try: 
                instructor = section["instructors"][0]['name']
            except:
                pass

            # Data unique to section list
            section_additions = [section["id"], section["activity"], instructor, section["credits"], section["course_quality"], section["instructor_quality"], section["difficulty"], section["work_required"], section["meetings"], section["associated_sections"]]
            
            section_list.append(common + section_additions)

        course_list.append(common + course_additions)


In [28]:
# saving to df

import pandas as pd

course_df = pd.DataFrame(course_list, columns=["id", "title", "description", "prerequisites", "course_quality", "instructor_quality", "difficulty", "work_required", "credits"]).drop_duplicates()
section_df = pd.DataFrame(section_list, columns=["id", "title", "description", "prerequisites", "section_id", "activity", "instructor", "credits", "course_quality", "instructor_quality", "difficulty", "work_required", "meetings", "associated_sections"]).drop_duplicates(subset=["section_id"])


In [29]:
# exporting

import json

course_json = course_df.to_json(orient="records")
with open('course_data.json', 'w') as f:
    f.write(course_json)

section_json = section_df.to_json(orient="records")
with open('section_data.json', 'w') as f:
    f.write(section_json)